In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

import tensorflow.keras as keras
from keras import models,layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.regularizers import l1

from tensorflow.keras.utils import to_categorical

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

In [ ]:
train.head()

In [ ]:
Name0=train['loss'].unique()
Name=sorted(Name0)
print(Name)

In [ ]:
train['loss'].value_counts()

In [ ]:
N=list(range(len(Name)))  
normal_mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

In [ ]:
target = train['loss'].map(normal_mapping)
trainX = train.drop(['loss','id'],axis=1)
testX = test.drop('id',axis=1)

In [ ]:
trainy2=to_categorical(target)
trainY=np.array(trainy2)

In [ ]:
print(trainX.shape)
print(trainY.shape)

In [ ]:
keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.Dense(256,activation='relu',input_dim=100),   ### input_dim=trainX.shape[1]
    keras.layers.Dense(128,activation='relu'),
    keras.layers.Dense(len(Name),activation='softmax')
])

early = keras.callbacks.EarlyStopping(patience=10)
model_check = keras.callbacks.ModelCheckpoint('model.h5',save_best_only=True)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
result = model.fit(trainX,trainY,epochs=10,batch_size=1)

In [ ]:
hist = pd.DataFrame(result.history)
hist.loc[:, ['loss','accuracy',]].plot()

In [ ]:
preds = model.predict(testX)
print(preds.shape)
print(preds[0])

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")
print(sample.shape)

In [ ]:
subm = sample
PRED=[]
for item in preds:
    value=np.argmax(item)      
    PRED+=[value]
subm['loss'] = PRED
subm.to_csv('submission.csv',index=False)
subm

In [ ]:
subm['loss'].value_counts()